In [1]:
from importlib.metadata import version
import torch

print("TORCH VERSION :", version("torch"))
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backend.mps.is_available() else "cpu"
)
print("Device  : ", device.upper())

TORCH VERSION : 2.2.2
Device  :  CUDA


In [5]:
# Inference
from src.models.gpt2.model import GPT2
from src.models.gpt2.config import GPT2Config


gpt_conf = {
    "block_size": 1024,
    "vocab_size": 32000,
    "n_layer": 8,
    "n_head": 12,
    "n_embd": 768,
    "dropout": 0.0,
    "bias": True,
    "device": device,
}

config = GPT2Config(**gpt_conf)
model = GPT2(config)

number of parameters: 81.25M


In [7]:
state_dict = torch.load(
    "/home/pranav-pc/projects/OpenTransformer/multiformer/model_checkpoint_epoch_1_iter_3000.pt"
)
unwanted_prefix = "_orig_mod."
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
model.eval()
model.to(device)
if compile:
    model = torch.compile(model)

In [9]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)
s = "What is the meaning of name 'Pranav'"
idx = encode(s)
x = torch.tensor(idx, dtype=torch.long, device=device)[None, ...]

In [12]:
# run generation
with torch.no_grad():
    for k in range(2):
        y = model.generate(x, max_new_tokens, temperature=temprature, top_k=top_k)
        print(decode(y[0].tolist()))
        print("---------------")

NameError: name 'max_new_tokens' is not defined